In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)
###Source = https://patentsview.org/download/data-download-dictionary


#####발명가(Inventor) : (21752644 , 7)
data_inv = pd.read_csv("./g_inventor_disambiguated.tsv", delimiter='\t', keep_default_na=False)

#####특허별 분류 모음 : (52528329, 7)
###G06N 항목은 컴퓨터 시스템을 기반으로 한 정보 처리 시스템
###ref : https://www.uspto.gov/web/patents/classification/cpc/html/defG06N.html
#G06N 3/00: 컴퓨터 시스템을 사용한 데이터 처리 시스템, 특히 인공지능에 관련된 것
#G06N 3/04: 신경망을 이용한 것
#G06N 5/00: 지능형 에이전트
#G06N 20/00: 기계 학습

###cs관련 patent : (69825, 7)
data_cpc = pd.read_csv("./g_cpc_current.tsv", delimiter='\t', keep_default_na=False)
only_cs = data_cpc.loc[data_cpc['cpc_subclass']=='G06N'].groupby('patent_id').head(1)

###Location 모음 : (92793, 9)
data_loc = pd.read_csv("./g_location_disambiguated.tsv", delimiter='\t', keep_default_na=False)

###Patent 모음 : (8701569, 9)
data_patent = pd.read_csv("./g_patent.tsv", delimiter='\t', keep_default_na=False)

###양수인(Assignee) : (8024360, 8)
data_assign = pd.read_csv("./g_assignee_disambiguated.tsv", delimiter='\t', keep_default_na=False)

/var/folders/92/p5xp12v11ws0b2gk_yzl42c80000gn/T/ipykernel_48041/1900733357.py:26: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data_patent = pd.read_csv("./g_patent.tsv", delimiter='\t', keep_default_na=False)


# 1. Cooperative Patent Classification(CPC) 활용

### 발명가 단위로 묶기
- 최종 row 수 : 235,064

In [2]:
###Inventor Data 중, 컴퓨터 사이언스 관련 특허만 Filter
only_cs['patent_id'] = only_cs['patent_id'].astype(str)
data = pd.merge(data_inv, only_cs, on = 'patent_id', how = 'inner')

###Inventor에 Location 데이터 추가
data = pd.merge(data, data_loc, on = 'location_id', how = 'inner') 

###Patent에 대한 정보 추가
data_patent['patent_id'] = data_patent['patent_id'].astype(str)
data = pd.merge(data, data_patent, on = 'patent_id', how = 'left')
data = data.sort_values(['patent_id', 'inventor_sequence'])
data.head()

,patent_id,inventor_sequence,inventor_id,disambig_inventor_name_first,disambig_inventor_name_last,gender_code,location_id,cpc_sequence,cpc_section,cpc_class,cpc_subclass,cpc_group,cpc_type,disambig_city,disambig_state,disambig_country,latitude,longitude,county,state_fips,county_fips,patent_type,patent_date,patent_title,patent_abstract,wipo_kind,num_claims,withdrawn,filename
163723,10001422,0,fl:ja_ln:jang-16,Jae-eun,Jang,U,3eb2eaaa-16c8-11ed-9b5f-1234bde3cd05,2,G,G06,G06N,G06N3/008,inventional,Daegu,,KR,35.87130,128.6018,,,,utility,2018-06-19,Method and device for sensing pain,A pain sensing device includes a sensor array ...,B2,9,0,ipg180619.xml
163725,10001422,1,fl:ky_ln:lee-253,Kyung Hwa,Lee,F,3eb2eaaa-16c8-11ed-9b5f-1234bde3cd05,2,G,G06,G06N,G06N3/008,inventional,Daegu,,KR,35.87130,128.6018,,,,utility,2018-06-19,Method and device for sensing pain,A pain sensing device includes a sensor array ...,B2,9,0,ipg180619.xml
207080,10001422,2,fl:mi_ln:sim-10,Minkyung,Sim,F,74f93c7c-16c8-11ed-9b5f-1234bde3cd05,2,G,G06,G06N,G06N3/008,inventional,Daega-myeon,,KR,35.02574,128.2814,,,,utility,2018-06-19,Method and device for sensing pain,A pain sensing device includes a sensor array ...,B2,9,0,ipg180619.xml
163724,10001422,3,fl:ch_ln:moon-68,Cheil,MOON,M,3eb2eaaa-16c8-11ed-9b5f-1234bde3cd05,2,G,G06,G06N,G06N3/008,inventional,Daegu,,KR,35.87130,128.6018,,,,utility,2018-06-19,Method and device for sensing pain,A pain sensing device includes a sensor array ...,B2,9,0,ipg180619.xml
163726,10001422,4,fl:ji_ln:choi-2,Ji Woong,Choi,M,3eb2eaaa-16c8-11ed-9b5f-1234bde3cd05,2,G,G06,G06N,G06N3/008,inventional,Daegu,,KR,35.87130,128.6018,,,,utility,2018-06-19,Method and device for sensing pain,A pain sensing device includes a sensor array ...,B2,9,0,ipg180619.xml


In [10]:
print("평균 공동 개발자 수 :", data[['patent_id', 'inventor_id', 'disambig_country']].groupby('patent_id').count().reset_index()['inventor_id'].mean())

평균 공동 개발자 수 : 3.368885704048728


In [9]:
print("평균 공동 개발 국가 수 :",data.groupby('patent_id')['disambig_country'].nunique().mean())

평균 공동 개발 국가 수 : 1.1395485489072017


In [31]:
unique_country_counts = data.groupby('patent_id')['disambig_country'].nunique()

# 고유한 값의 갯수가 1보다 큰 patent_id만 가져오기
patent_ids_with_multiple_countries = unique_country_counts[unique_country_counts > 1]

# loc을 사용하여 중복된 disambig_country 값이 1보다 큰 patent_id만 가져오기
filtered_data = data.loc[data['patent_id'].isin(patent_ids_with_multiple_countries.index)]
print('전체 특허 수 :' ,len(data['patent_id'].unique()))
print('국가 간 공동 개발 특허 수 :' ,len(filtered_data['patent_id'].unique()))


전체 특허 수 : 69775
국가 간 공동 개발 특허 수 : 8594


### 양수자 단위로 묶기
- row : 68,822

In [12]:
###Assignee Type
"""
classification of assignee 
1 - Unassigned
2 - US Company or Corporation
3 - Foreign Company or Corporation
4 - US Individual
5 - Foreign Individual
6 - US Federal Government
7 - Foreign Government
8 - US County Government
9 - US State Government. Note: A "1" appearing before any of these codes signifies part interest
"""

###Inventor Data 중, 컴퓨터 사이언스 관련 특허만 Filter
only_cs['patent_id'] = only_cs['patent_id'].astype(str)
data_2 = pd.merge(data_assign, only_cs, on = 'patent_id', how = 'inner')

###Inventor에 Location 데이터 추가
data_2 = pd.merge(data_2, data_loc, on = 'location_id', how = 'inner') 

###Patent에 대한 정보 추가
data_patent['patent_id'] = data_patent['patent_id'].astype(str)
data_2 = pd.merge(data_2, data_patent, on = 'patent_id', how = 'left')
data_2 = data_2.sort_values(['patent_id', 'assignee_sequence'])
data_2.head()

,patent_id,assignee_sequence,assignee_id,disambig_assignee_individual_name_first,disambig_assignee_individual_name_last,disambig_assignee_organization,assignee_type,location_id,cpc_sequence,cpc_section,cpc_class,cpc_subclass,cpc_group,cpc_type,disambig_city,disambig_state,disambig_country,latitude,longitude,county,state_fips,county_fips,patent_type,patent_date,patent_title,patent_abstract,wipo_kind,num_claims,withdrawn,filename
67842,10001422,0,eed40e76-c236-403d-9b6f-8b325d6b9c86,,,Daegu Gyeongbuk Institute fo Science and Techn...,3,7514cfb6-16c8-11ed-9b5f-1234bde3cd05,2,G,G06,G06N,G06N3/008,inventional,Habin-myeon,,KR,35.895515,128.424642,,,,utility,2018-06-19,Method and device for sensing pain,A pain sensing device includes a sensor array ...,B2,9,0,ipg180619.xml
41746,10001763,0,c7d317d8-eb9a-4ff6-8ed4-28f26960c794,,,LENOVO (BEIJING) LIMITED,3,3b444e27-16c8-11ed-9b5f-1234bde3cd05,2,G,G06,G06N,G06N5/025,additional,Beijing,,CN,39.906217,116.391276,,,,utility,2018-06-19,Control device and method for controlling inte...,A method for controlling intelligent device ba...,B2,11,0,ipg180619.xml
14388,10001871,0,9407ac39-e208-4171-9508-731f436be923,,,Intel Corporation,2,c1ee6332-16c7-11ed-9b5f-1234bde3cd05,3,G,G06,G06N,G06N20/00,additional,Santa Clara,CA,US,37.354113,-121.955174,Santa Clara,06,085,utility,2018-06-19,Mobile device rejection of unintentional touch...,Mobile device rejection of unintentional senso...,B2,20,0,ipg180619.xml
26338,10001902,0,343fce45-0ef5-4700-a0d2-13135e222089,,,"GROUPON, INC.",2,e1c05366-16c7-11ed-9b5f-1234bde3cd05,5,G,G06,G06N,G06N5/02,inventional,Chicago,IL,US,41.875562,-87.624421,Cook,17,031,utility,2018-06-19,Learning user interface,"Provided herein are method, apparatus, and com...",B2,57,0,ipg180619.xml
17851,10001978,0,7baa8688-f6a4-4b26-bde4-61786f6dae06,,,Oracle International Corporation,2,c450eb14-16c7-11ed-9b5f-1234bde3cd05,2,G,G06,G06N,G06N5/01,inventional,Redwood Shores,CA,US,37.531880,-122.248022,San Mateo County,06,081,utility,2018-06-19,Type inference optimization,Operations include (a) identifying bounds corr...,B2,22,0,ipg180619.xml


In [17]:
print('평균 공동 특허 소유 수 :',data_2.groupby('patent_id')['disambig_assignee_organization'].count().mean())

평균 공동 특허 소유 수 : 1.0248994787788532


# 2. AI Patent Dataset 활용
- 전체 Row : 13,244,037
- 머신러닝 모델로 예측해서, Artificial Intelligent와 연동되었다고 생각되는 Application Row : 1,517,174
- Patent Id와 연동 되는 Row : 1,152,046

In [21]:
###Artificial Intelligence Patent Dataset
#Source: https://www.uspto.gov/ip-policy/economic-research/research-datasets/artificial-intelligence-patent-dataset
data_ai_model = pd.read_csv("./ai_model_predictions.tsv", delimiter='\t', keep_default_na=False)
data_ai_model = data_ai_model.loc[data_ai_model['predict50_any_ai']==1] #AI 여러 분야 중, 하나라도 연동되는 것.

###Application과 Patent 연결 데이터셋
data_app = pd.read_csv("./g_application.tsv", delimiter='\t', keep_default_na=False)

###두 개 Merge
data_ai = pd.merge(data_ai_model, data_app, left_on = 'appl_id', right_on = 'application_id', how = 'inner')
data_ai = data_ai[['patent_id', 'application_id']].drop_duplicates()


/var/folders/92/p5xp12v11ws0b2gk_yzl42c80000gn/T/ipykernel_48041/1615027742.py:3: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  data_ai_model = pd.read_csv("./ai_model_predictions.tsv", delimiter='\t', keep_default_na=False)
/var/folders/92/p5xp12v11ws0b2gk_yzl42c80000gn/T/ipykernel_48041/1615027742.py:7: DtypeWarning: Columns (0,2,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  data_app = pd.read_csv("./g_application.tsv", delimiter='\t', keep_default_na=False)


### 발명가 단위

In [22]:
data3 = pd.merge(data_inv, data_ai, on = 'patent_id', how = 'inner')
data3 = pd.merge(data3, data_loc, on = 'location_id', how = 'inner') 
data3 = pd.merge(data3, data_patent, on = 'patent_id', how = 'left')
data3 = data3.sort_values(['patent_id', 'inventor_sequence'])
data3.head()

,patent_id,inventor_sequence,inventor_id,disambig_inventor_name_first,disambig_inventor_name_last,gender_code,location_id,application_id,disambig_city,disambig_state,disambig_country,latitude,longitude,county,state_fips,county_fips,patent_type,patent_date,patent_title,patent_abstract,wipo_kind,num_claims,withdrawn,filename
1873342,10000080,0,fl:xa_ln:farinavargas-1,Xavier,Farina Vargas,M,bf69d3e3-16c7-11ed-9b5f-1234bde3cd05,15547729,Sant Cugat del Vallès,,ES,41.472843,2.081781,,,,utility,2018-06-19,Random wave mask generation,Example implementations relate to random wave ...,B2,15,0,ipg180619.xml
1873341,10000080,1,fl:m._ln:borrellbayona-1,M. Isabel,Borrell Bayona,F,bf69d3e3-16c7-11ed-9b5f-1234bde3cd05,15547729,Sant Cugat del Vallès,,ES,41.472843,2.081781,,,,utility,2018-06-19,Random wave mask generation,Example implementations relate to random wave ...,B2,15,0,ipg180619.xml
1873343,10000080,2,fl:an_ln:martinezbarambio-2,Angel,Martinez Barambio,M,bf69d3e3-16c7-11ed-9b5f-1234bde3cd05,15547729,Sant Cugat del Vallès,,ES,41.472843,2.081781,,,,utility,2018-06-19,Random wave mask generation,Example implementations relate to random wave ...,B2,15,0,ipg180619.xml
2042154,10000100,0,fl:da_ln:weston-6,David Alan,Weston,M,f4421c0f-16c7-11ed-9b5f-1234bde3cd05,13977418,Hendersonville,NC,US,35.318728,-82.460953,Henderson,37,089,utility,2018-06-19,Piezoelectric based system and method for dete...,Systems and methods of determining tire load f...,B2,14,0,ipg180619.xml
1263375,10000103,0,fl:da_ln:hrovat-1,Davorin David,Hrovat,M,4c7a9f37-16c8-11ed-9b5f-1234bde3cd05,15584306,Ann Arbor,MI,US,42.268157,-83.731229,Washtenaw,26,161,utility,2018-06-19,Method of controlling a vehicle active suspens...,A method of controlling a vehicle active suspe...,B2,20,0,ipg180619.xml


In [23]:
print("평균 공동 개발자 수 :", data3[['patent_id', 'inventor_id', 'disambig_country']].groupby('patent_id').count().reset_index()['inventor_id'].mean())

평균 공동 개발자 수 : 2.9246470550770995


In [27]:
print("평균 공동 개발 국가 수 :",data3.groupby('patent_id')['disambig_country'].nunique().mean())

평균 공동 개발 국가 수 : 1.1058714466672799


In [30]:
unique_country_counts = data3.groupby('patent_id')['disambig_country'].nunique()

# 고유한 값의 갯수가 1보다 큰 patent_id만 가져오기
patent_ids_with_multiple_countries = unique_country_counts[unique_country_counts > 1]

# loc을 사용하여 중복된 disambig_country 값이 1보다 큰 patent_id만 가져오기
filtered_data = data3.loc[data3['patent_id'].isin(patent_ids_with_multiple_countries.index)]
print('전체 특허 수 :' ,len(data3['patent_id'].unique()))
print('국가 간 공동 개발 특허 수 :' ,len(filtered_data['patent_id'].unique()))


전체 특허 수 : 706555
국가 간 공동 개발 특허 수 : 67610


### 양수자 단위

In [25]:
data_4 = pd.merge(data_assign, data_ai, on = 'patent_id', how = 'inner')

###Inventor에 Location 데이터 추가
data_4 = pd.merge(data_4, data_loc, on = 'location_id', how = 'inner') 

###Patent에 대한 정보 추가
data_patent['patent_id'] = data_patent['patent_id'].astype(str)
data_4 = pd.merge(data_4, data_patent, on = 'patent_id', how = 'left')
data_4 = data_4.sort_values(['patent_id', 'assignee_sequence'])
data_4.head()

,patent_id,assignee_sequence,assignee_id,disambig_assignee_individual_name_first,disambig_assignee_individual_name_last,disambig_assignee_organization,assignee_type,location_id,application_id,disambig_city,disambig_state,disambig_country,latitude,longitude,county,state_fips,county_fips,patent_type,patent_date,patent_title,patent_abstract,wipo_kind,num_claims,withdrawn,filename
276037,10000080,0,3355f960-ac8a-4684-b91d-b60fb9e76538,,,"Hewlett-Packard Development Company, L.P.",2,4e49a0a0-16c8-11ed-9b5f-1234bde3cd05,15547729,Houston,TX,US,29.758938,-95.367697,Harris,48,201,utility,2018-06-19,Random wave mask generation,Example implementations relate to random wave ...,B2,15,0,ipg180619.xml
576654,10000100,0,25defe88-2410-4ad2-8363-8438087e1f2d,,,COMPAGNIE GENERALE DES ETABLISSEMENTS MICHELIN,3,442df657-16c8-11ed-9b5f-1234bde3cd05,13977418,Clermont-Ferrand,,FR,45.777455,3.081943,,,,utility,2018-06-19,Piezoelectric based system and method for dete...,Systems and methods of determining tire load f...,B2,14,0,ipg180619.xml
469281,10000103,0,bb295522-0c7e-4d68-bdc1-29086d9fbdf2,,,"Ford Global Technologies, LLC",2,ec485a6e-16c7-11ed-9b5f-1234bde3cd05,15584306,Dearborn,MI,US,42.322260,-83.176315,Wayne,26,163,utility,2018-06-19,Method of controlling a vehicle active suspens...,A method of controlling a vehicle active suspe...,B2,20,0,ipg180619.xml
689316,10000130,0,7795d1dd-4162-41ac-a3f1-8efb5c379e15,,,XI'AN TGOOD INTELLIGENT CHARGING TECHNOLOGY CO...,3,8800932e-16c8-11ed-9b5f-1234bde3cd05,15144816,Zhiluo,,CN,35.958491,109.001448,,,,utility,2018-06-19,Charging management system and method for char...,A charging management system (CMS) and a charg...,B2,10,0,ipg180619.xml
389344,10000196,0,ef14c76a-a594-4696-863e-78b8e0c4607d,,,Hyundai Motor Company,3,1aeb3c64-16c8-11ed-9b5f-1234bde3cd05,14929707,Seoul,,KR,37.566679,126.978291,,,,utility,2018-06-19,Apparatus and method for controlling torque re...,A method for controlling torque reduction of a...,B2,11,0,ipg180619.xml


In [26]:
print('평균 공동 특허 소유 수 :',data_4.groupby('patent_id')['disambig_assignee_organization'].count().mean())

평균 공동 특허 소유 수 : 1.0222129061502863
